In [ ]:
from covidcomp.data import RawRepresentation, DerivedRepresentation
from covidcomp.model import L2RegularisedLogisticRegression
from covidcomp.experiment import ExperimentRunner
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from fomlads.model.classification import logistic_regression_fit, logistic_regression_predict, shared_covariance_model_fit, shared_covariance_model_predict
from fomlads.evaluate.eval_classification import eval_accuracy
import numpy as np

## Create raw data representation from `./covid.csv` and auxiliary datasets

In [ ]:
raw = RawRepresentation("./covid.csv")

## Prepare the model

In [ ]:
model = L2RegularisedLogisticRegression(lambda_=1e3)

# Flat comparison
## Get flat data representation and fit model on Flat and test

In [ ]:
flat_raw = raw.get_representation()
ExperimentRunner.run_train_and_test(flat_raw, model, label="Flat")

# Partition by Continent

## Generate raw and partitioned representations by continent. For each continent, get the derived representation and conduct experiment

In [ ]:
partitioned_by_continent_dict = raw.get_representation("continent")
ExperimentRunner.run_train_and_test(partitioned_by_continent_dict, model, label="Continent")

# Partition by Income Group

## Generate raw and partitioned representations by Income Group. For each Income Group, get the derived representation and conduct experiment

In [ ]:
partitioned_by_income = raw.get_representation("income_group")
ExperimentRunner.run_train_and_test(partitioned_by_income, model, label="Income Group")